In [1]:
## This file implements neural networks and logistic regression on p002ypresabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 85.51% for combination data, and 85.73% for over-sampling data.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## For the new neural networks on these new datasets, the accuracy is 88.8% for combination data and 82.99% for over-sampling data.
## For logistic regression, the accuracies are 100% for both combination and over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracies are again 100% for both combination and over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.68% for combination data, and 98.83% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p002ypresabsSTCC_quant.csv')
df.shape

(255, 1761)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.194875
1      0.265250
2      0.440625
3      0.175500
4      0.173625
5      0.270375
6      0.171000
7      0.182500
8      0.278875
9      0.174375
10     0.176250
11     0.411125
12     0.178250
13     0.191500
14     0.164875
15     0.181875
16     0.394125
17     0.364875
18     0.409875
19     0.191875
20     0.236125
21     0.277000
22     0.169875
23     0.171625
24     0.523625
25     0.169500
26     0.381000
27     0.170000
28     0.160750
29     0.389750
         ...   
225    0.230500
226    0.279800
227    0.319500
228    0.424100
229    0.460500
230    0.304750
231    0.193875
232    0.243000
233    0.195750
234    0.184750
235    0.329125
236    0.160000
237    0.157500
238    0.176375
239    0.326750
240    0.355750
241    0.281375
242    0.181875
243    0.174125
244    0.359250
245    0.161375
246    0.185875
247    0.172750
248    0.167750
249    0.179875
250    0.365375
251    0.223500
252    0.170750
253    0.261125
254    0.231000
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno,ST,CC
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.194875,5,5
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.265250,8,8
2,115,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0.440625,5,5
3,120335,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0.175500,5,5
4,120337,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0.173625,5,5


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    244
1     11
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 1761)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 1760)

In [13]:
df_clean.head()

,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno,ST,CC
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,5,5
3,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,5,5
4,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,5,5


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1759) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 224), (1, 233)]


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 244), (1, 244)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 319 samples, validate on 138 samples
Epoch 1/1000
319/319 [==============================] - 0s 375us/step - loss: 11.9016 - accuracy: 0.6771 - val_loss: 0.5614 - val_accuracy: 0.7464
Epoch 2/1000
319/319 [==============================] - 0s 98us/step - loss: 0.5344 - accuracy: 0.7994 - val_loss: 0.5526 - val_accuracy: 0.7609
Epoch 3/1000
319/319 [==============================] - 0s 98us/step - loss: 0.5245 - accuracy: 0.8150 - val_loss: 0.5413 - val_accuracy: 0.7971
Epoch 4/1000
319/319 [==============================] - 0s 107us/step - loss: 0.5145 - accuracy: 0.8182 - val_loss: 0.5341 - val_accuracy: 0.7609
Epoch 5/1000
319/319 [==============================] - 0s 103us/step - loss: 0.5080 - accuracy: 0.8150 - val_loss: 0.5284 - val_accuracy: 0.8116
Epoch 6/1000
319/319 [==============================] - 0s 104us/step - loss: 0.5002 - accuracy: 0.8339 - val_loss: 0.5212 - val_accuracy: 0.7899
Epoch 7/1000
319/319 [==============================] - 0s 105us/step - loss: 0

319/319 [==============================] - 0s 87us/step - loss: 0.3465 - accuracy: 0.8840 - val_loss: 0.3719 - val_accuracy: 0.8551
Epoch 57/1000
319/319 [==============================] - 0s 89us/step - loss: 0.3450 - accuracy: 0.8840 - val_loss: 0.3696 - val_accuracy: 0.8551
Epoch 58/1000
319/319 [==============================] - 0s 91us/step - loss: 0.3433 - accuracy: 0.8840 - val_loss: 0.3691 - val_accuracy: 0.8551
Epoch 59/1000
319/319 [==============================] - 0s 89us/step - loss: 0.3426 - accuracy: 0.8840 - val_loss: 0.3697 - val_accuracy: 0.8551
Epoch 60/1000
319/319 [==============================] - 0s 87us/step - loss: 0.3420 - accuracy: 0.8840 - val_loss: 0.3676 - val_accuracy: 0.8551
Epoch 61/1000
319/319 [==============================] - 0s 83us/step - loss: 0.3405 - accuracy: 0.8840 - val_loss: 0.3690 - val_accuracy: 0.8551
Epoch 62/1000
319/319 [==============================] - 0s 84us/step - loss: 0.3394 - accuracy: 0.8840 - val_loss: 0.3659 - val_accuracy:

319/319 [==============================] - 0s 118us/step - loss: 0.3134 - accuracy: 0.8809 - val_loss: 0.3320 - val_accuracy: 0.8551
Epoch 113/1000
319/319 [==============================] - 0s 108us/step - loss: 0.3093 - accuracy: 0.8809 - val_loss: 0.3527 - val_accuracy: 0.8478
Epoch 114/1000
319/319 [==============================] - 0s 108us/step - loss: 0.4309 - accuracy: 0.8558 - val_loss: 0.3847 - val_accuracy: 0.8261
Epoch 115/1000
319/319 [==============================] - 0s 101us/step - loss: 0.3482 - accuracy: 0.8558 - val_loss: 0.3827 - val_accuracy: 0.8551
Epoch 116/1000
319/319 [==============================] - 0s 100us/step - loss: 0.3461 - accuracy: 0.8652 - val_loss: 0.3817 - val_accuracy: 0.8551
Epoch 117/1000
319/319 [==============================] - 0s 103us/step - loss: 0.3446 - accuracy: 0.8809 - val_loss: 0.3801 - val_accuracy: 0.8551
Epoch 118/1000
319/319 [==============================] - 0s 113us/step - loss: 0.3432 - accuracy: 0.8777 - val_loss: 0.3790 - 

Epoch 168/1000
319/319 [==============================] - 0s 88us/step - loss: 0.2700 - accuracy: 0.8809 - val_loss: 0.3122 - val_accuracy: 0.8551
Epoch 169/1000
319/319 [==============================] - 0s 79us/step - loss: 0.2712 - accuracy: 0.8809 - val_loss: 0.3085 - val_accuracy: 0.8478
Epoch 170/1000
319/319 [==============================] - 0s 88us/step - loss: 0.2806 - accuracy: 0.8871 - val_loss: 0.3014 - val_accuracy: 0.8478
Epoch 171/1000
319/319 [==============================] - 0s 101us/step - loss: 0.2678 - accuracy: 0.8809 - val_loss: 0.2997 - val_accuracy: 0.8478
Epoch 172/1000
319/319 [==============================] - 0s 95us/step - loss: 0.2865 - accuracy: 0.8809 - val_loss: 0.3072 - val_accuracy: 0.8478
Epoch 173/1000
319/319 [==============================] - 0s 87us/step - loss: 0.2677 - accuracy: 0.8809 - val_loss: 0.2973 - val_accuracy: 0.8478
Epoch 174/1000
319/319 [==============================] - 0s 87us/step - loss: 0.2611 - accuracy: 0.8809 - val_loss: 

Epoch 224/1000
319/319 [==============================] - 0s 83us/step - loss: 0.2398 - accuracy: 0.8871 - val_loss: 0.2757 - val_accuracy: 0.8551
Epoch 225/1000
319/319 [==============================] - 0s 86us/step - loss: 0.2416 - accuracy: 0.8871 - val_loss: 0.2743 - val_accuracy: 0.8551
Epoch 226/1000
319/319 [==============================] - 0s 85us/step - loss: 0.2448 - accuracy: 0.8871 - val_loss: 0.2719 - val_accuracy: 0.8551
Epoch 227/1000
319/319 [==============================] - 0s 82us/step - loss: 0.2416 - accuracy: 0.8871 - val_loss: 0.2718 - val_accuracy: 0.8551
Epoch 228/1000
319/319 [==============================] - 0s 87us/step - loss: 0.2395 - accuracy: 0.8871 - val_loss: 0.2727 - val_accuracy: 0.8551
Epoch 229/1000
319/319 [==============================] - 0s 86us/step - loss: 0.2411 - accuracy: 0.8871 - val_loss: 0.2706 - val_accuracy: 0.8551
Epoch 230/1000
319/319 [==============================] - 0s 87us/step - loss: 0.2387 - accuracy: 0.8871 - val_loss: 0

...

319/319 [==============================] - 0s 149us/step - loss: 0.2441 - accuracy: 0.8871 - val_loss: 0.2914 - val_accuracy: 0.8551
Epoch 776/1000
319/319 [==============================] - 0s 185us/step - loss: 0.2426 - accuracy: 0.8871 - val_loss: 0.2912 - val_accuracy: 0.8551
Epoch 777/1000
319/319 [==============================] - 0s 146us/step - loss: 0.2425 - accuracy: 0.8871 - val_loss: 0.2907 - val_accuracy: 0.8551
Epoch 778/1000
319/319 [==============================] - 0s 176us/step - loss: 0.2424 - accuracy: 0.8871 - val_loss: 0.2873 - val_accuracy: 0.8551
Epoch 779/1000
319/319 [==============================] - 0s 275us/step - loss: 0.2417 - accuracy: 0.8871 - val_loss: 0.2843 - val_accuracy: 0.8551
Epoch 780/1000
319/319 [==============================] - 0s 166us/step - loss: 0.2417 - accuracy: 0.8871 - val_loss: 0.2827 - val_accuracy: 0.8551
Epoch 781/1000
319/319 [==============================] - 0s 155us/step - loss: 0.2411 - accuracy: 0.8871 - val_loss: 0.2815 - 

319/319 [==============================] - 0s 144us/step - loss: 0.2337 - accuracy: 0.8871 - val_loss: 0.2665 - val_accuracy: 0.8551
Epoch 831/1000
319/319 [==============================] - 0s 186us/step - loss: 0.2329 - accuracy: 0.8871 - val_loss: 0.2662 - val_accuracy: 0.8551
Epoch 832/1000
319/319 [==============================] - 0s 129us/step - loss: 0.2320 - accuracy: 0.8871 - val_loss: 0.2661 - val_accuracy: 0.8551
Epoch 833/1000
319/319 [==============================] - 0s 162us/step - loss: 0.2334 - accuracy: 0.8871 - val_loss: 0.2663 - val_accuracy: 0.8551
Epoch 834/1000
319/319 [==============================] - 0s 107us/step - loss: 0.2325 - accuracy: 0.8871 - val_loss: 0.2659 - val_accuracy: 0.8551
Epoch 835/1000
319/319 [==============================] - 0s 141us/step - loss: 0.2314 - accuracy: 0.8871 - val_loss: 0.2698 - val_accuracy: 0.8551
Epoch 836/1000
319/319 [==============================] - 0s 138us/step - loss: 0.2351 - accuracy: 0.8871 - val_loss: 0.2653 - 

319/319 [==============================] - 0s 87us/step - loss: 0.2296 - accuracy: 0.8871 - val_loss: 0.2612 - val_accuracy: 0.8551
Epoch 886/1000
319/319 [==============================] - 0s 96us/step - loss: 0.2295 - accuracy: 0.8871 - val_loss: 0.2613 - val_accuracy: 0.8551
Epoch 887/1000
319/319 [==============================] - 0s 102us/step - loss: 0.2309 - accuracy: 0.8871 - val_loss: 0.2631 - val_accuracy: 0.8551
Epoch 888/1000
319/319 [==============================] - 0s 98us/step - loss: 0.2306 - accuracy: 0.8871 - val_loss: 0.2605 - val_accuracy: 0.8551
Epoch 889/1000
319/319 [==============================] - 0s 96us/step - loss: 0.2304 - accuracy: 0.8871 - val_loss: 0.2610 - val_accuracy: 0.8551
Epoch 890/1000
319/319 [==============================] - 0s 96us/step - loss: 0.2298 - accuracy: 0.8871 - val_loss: 0.2620 - val_accuracy: 0.8551
Epoch 891/1000
319/319 [==============================] - 0s 109us/step - loss: 0.2297 - accuracy: 0.8871 - val_loss: 0.2601 - val_a

319/319 [==============================] - 0s 156us/step - loss: 0.2296 - accuracy: 0.8871 - val_loss: 0.2602 - val_accuracy: 0.8551
Epoch 941/1000
319/319 [==============================] - 0s 131us/step - loss: 0.2292 - accuracy: 0.8871 - val_loss: 0.2579 - val_accuracy: 0.8551
Epoch 942/1000
319/319 [==============================] - 0s 153us/step - loss: 0.2278 - accuracy: 0.8871 - val_loss: 0.2570 - val_accuracy: 0.8551
Epoch 943/1000
319/319 [==============================] - 0s 115us/step - loss: 0.2279 - accuracy: 0.8871 - val_loss: 0.2578 - val_accuracy: 0.8551
Epoch 944/1000
319/319 [==============================] - 0s 105us/step - loss: 0.2278 - accuracy: 0.8871 - val_loss: 0.2568 - val_accuracy: 0.8551
Epoch 945/1000
319/319 [==============================] - 0s 101us/step - loss: 0.2300 - accuracy: 0.8871 - val_loss: 0.2613 - val_accuracy: 0.8551
Epoch 946/1000
319/319 [==============================] - 0s 90us/step - loss: 0.2306 - accuracy: 0.8871 - val_loss: 0.2592 - v

319/319 [==============================] - 0s 207us/step - loss: 0.2272 - accuracy: 0.8871 - val_loss: 0.2553 - val_accuracy: 0.8551
Epoch 996/1000
319/319 [==============================] - 0s 236us/step - loss: 0.2271 - accuracy: 0.8871 - val_loss: 0.2565 - val_accuracy: 0.8551
Epoch 997/1000
319/319 [==============================] - 0s 201us/step - loss: 0.2274 - accuracy: 0.8871 - val_loss: 0.2554 - val_accuracy: 0.8551
Epoch 998/1000
319/319 [==============================] - 0s 124us/step - loss: 0.2270 - accuracy: 0.8871 - val_loss: 0.2550 - val_accuracy: 0.8551
Epoch 999/1000
319/319 [==============================] - 0s 131us/step - loss: 0.2271 - accuracy: 0.8871 - val_loss: 0.2559 - val_accuracy: 0.8551
Epoch 1000/1000
319/319 [==============================] - 0s 119us/step - loss: 0.2272 - accuracy: 0.8871 - val_loss: 0.2551 - val_accuracy: 0.8551


In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

138/138 [==============================] - 0s 250us/step
combination test accuracy: 85.51%


In [25]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [26]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 341 samples, validate on 147 samples
Epoch 1/1000
341/341 [==============================] - 0s 1ms/step - loss: 5.2075 - accuracy: 0.6422 - val_loss: 0.6015 - val_accuracy: 0.7211
Epoch 2/1000
341/341 [==============================] - 0s 133us/step - loss: 0.5940 - accuracy: 0.7478 - val_loss: 0.5878 - val_accuracy: 0.6803
Epoch 3/1000
341/341 [==============================] - 0s 147us/step - loss: 0.5850 - accuracy: 0.7449 - val_loss: 0.5819 - val_accuracy: 0.5986
Epoch 4/1000
341/341 [==============================] - 0s 136us/step - loss: 0.5802 - accuracy: 0.7273 - val_loss: 0.5830 - val_accuracy: 0.6871
Epoch 5/1000
341/341 [==============================] - 0s 131us/step - loss: 0.5726 - accuracy: 0.7507 - val_loss: 0.5645 - val_accuracy: 0.7823
Epoch 6/1000
341/341 [==============================] - 0s 159us/step - loss: 0.5604 - accuracy: 0.7801 - val_loss: 0.5622 - val_accuracy: 0.7483
Epoch 7/1000
341/341 [==============================] - 0s 159us/step - loss: 0.

341/341 [==============================] - 0s 186us/step - loss: 0.3804 - accuracy: 0.8534 - val_loss: 0.4190 - val_accuracy: 0.8367
Epoch 57/1000
341/341 [==============================] - 0s 164us/step - loss: 0.4020 - accuracy: 0.8446 - val_loss: 0.3883 - val_accuracy: 0.8299
Epoch 58/1000
341/341 [==============================] - 0s 163us/step - loss: 0.3723 - accuracy: 0.8651 - val_loss: 0.3672 - val_accuracy: 0.8571
Epoch 59/1000
341/341 [==============================] - 0s 154us/step - loss: 0.3689 - accuracy: 0.8622 - val_loss: 0.3664 - val_accuracy: 0.8571
Epoch 60/1000
341/341 [==============================] - 0s 177us/step - loss: 0.3698 - accuracy: 0.8622 - val_loss: 0.3707 - val_accuracy: 0.8503
Epoch 61/1000
341/341 [==============================] - 0s 138us/step - loss: 0.3636 - accuracy: 0.8710 - val_loss: 0.3697 - val_accuracy: 0.8571
Epoch 62/1000
341/341 [==============================] - 0s 135us/step - loss: 0.3629 - accuracy: 0.8710 - val_loss: 0.3773 - val_ac

Epoch 112/1000
341/341 [==============================] - 0s 156us/step - loss: 0.3267 - accuracy: 0.8710 - val_loss: 0.3351 - val_accuracy: 0.8571
Epoch 113/1000
341/341 [==============================] - 0s 109us/step - loss: 0.3283 - accuracy: 0.8622 - val_loss: 0.3404 - val_accuracy: 0.8571
Epoch 114/1000
341/341 [==============================] - 0s 146us/step - loss: 0.3220 - accuracy: 0.8710 - val_loss: 0.3337 - val_accuracy: 0.8571
Epoch 115/1000
341/341 [==============================] - 0s 113us/step - loss: 0.3283 - accuracy: 0.8710 - val_loss: 0.3333 - val_accuracy: 0.8571
Epoch 116/1000
341/341 [==============================] - 0s 194us/step - loss: 0.3236 - accuracy: 0.8710 - val_loss: 0.3325 - val_accuracy: 0.8571
Epoch 117/1000
341/341 [==============================] - 0s 150us/step - loss: 0.3297 - accuracy: 0.8592 - val_loss: 0.3377 - val_accuracy: 0.8571
Epoch 118/1000
341/341 [==============================] - 0s 114us/step - loss: 0.3232 - accuracy: 0.8768 - val_

341/341 [==============================] - 0s 98us/step - loss: 0.3115 - accuracy: 0.8680 - val_loss: 0.3393 - val_accuracy: 0.8639
Epoch 168/1000
341/341 [==============================] - 0s 90us/step - loss: 0.3109 - accuracy: 0.8768 - val_loss: 0.3218 - val_accuracy: 0.8571
Epoch 169/1000
341/341 [==============================] - 0s 83us/step - loss: 0.3109 - accuracy: 0.8710 - val_loss: 0.3227 - val_accuracy: 0.8571
Epoch 170/1000
341/341 [==============================] - 0s 85us/step - loss: 0.3123 - accuracy: 0.8710 - val_loss: 0.3277 - val_accuracy: 0.8571
Epoch 171/1000
341/341 [==============================] - 0s 79us/step - loss: 0.3102 - accuracy: 0.8710 - val_loss: 0.3241 - val_accuracy: 0.8571
Epoch 172/1000
341/341 [==============================] - 0s 90us/step - loss: 0.3119 - accuracy: 0.8651 - val_loss: 0.3228 - val_accuracy: 0.8571
Epoch 173/1000
341/341 [==============================] - 0s 86us/step - loss: 0.3110 - accuracy: 0.8710 - val_loss: 0.3246 - val_acc

341/341 [==============================] - 0s 153us/step - loss: 0.3024 - accuracy: 0.8739 - val_loss: 0.3182 - val_accuracy: 0.8571
Epoch 223/1000
341/341 [==============================] - 0s 203us/step - loss: 0.3050 - accuracy: 0.8739 - val_loss: 0.3206 - val_accuracy: 0.8571
Epoch 224/1000
341/341 [==============================] - 0s 167us/step - loss: 0.3059 - accuracy: 0.8710 - val_loss: 0.3218 - val_accuracy: 0.8571
Epoch 225/1000
341/341 [==============================] - 0s 152us/step - loss: 0.3060 - accuracy: 0.8710 - val_loss: 0.3196 - val_accuracy: 0.8571
Epoch 226/1000
341/341 [==============================] - 0s 173us/step - loss: 0.3049 - accuracy: 0.8710 - val_loss: 0.3209 - val_accuracy: 0.8571
Epoch 227/1000
341/341 [==============================] - 0s 160us/step - loss: 0.3051 - accuracy: 0.8680 - val_loss: 0.3205 - val_accuracy: 0.8571
Epoch 228/1000
341/341 [==============================] - 0s 163us/step - loss: 0.3036 - accuracy: 0.8739 - val_loss: 0.3180 - 

...

341/341 [==============================] - 0s 95us/step - loss: 0.2870 - accuracy: 0.8827 - val_loss: 0.3098 - val_accuracy: 0.8639
Epoch 773/1000
341/341 [==============================] - 0s 95us/step - loss: 0.2874 - accuracy: 0.8827 - val_loss: 0.3071 - val_accuracy: 0.8639
Epoch 774/1000
341/341 [==============================] - 0s 94us/step - loss: 0.2895 - accuracy: 0.8798 - val_loss: 0.3067 - val_accuracy: 0.8639
Epoch 775/1000
341/341 [==============================] - 0s 95us/step - loss: 0.2884 - accuracy: 0.8798 - val_loss: 0.3077 - val_accuracy: 0.8639
Epoch 776/1000
341/341 [==============================] - 0s 92us/step - loss: 0.2866 - accuracy: 0.8827 - val_loss: 0.3107 - val_accuracy: 0.8571
Epoch 777/1000
341/341 [==============================] - 0s 90us/step - loss: 0.2873 - accuracy: 0.8827 - val_loss: 0.3071 - val_accuracy: 0.8639
Epoch 778/1000
341/341 [==============================] - 0s 95us/step - loss: 0.2901 - accuracy: 0.8768 - val_loss: 0.3081 - val_acc

Epoch 828/1000
341/341 [==============================] - 0s 83us/step - loss: 0.2867 - accuracy: 0.8827 - val_loss: 0.3083 - val_accuracy: 0.8639
Epoch 829/1000
341/341 [==============================] - 0s 85us/step - loss: 0.2865 - accuracy: 0.8827 - val_loss: 0.3068 - val_accuracy: 0.8639
Epoch 830/1000
341/341 [==============================] - 0s 85us/step - loss: 0.2856 - accuracy: 0.8827 - val_loss: 0.3066 - val_accuracy: 0.8639
Epoch 831/1000
341/341 [==============================] - 0s 88us/step - loss: 0.2854 - accuracy: 0.8827 - val_loss: 0.3175 - val_accuracy: 0.8639
Epoch 832/1000
341/341 [==============================] - 0s 97us/step - loss: 0.2861 - accuracy: 0.8827 - val_loss: 0.3083 - val_accuracy: 0.8639
Epoch 833/1000
341/341 [==============================] - 0s 96us/step - loss: 0.2865 - accuracy: 0.8827 - val_loss: 0.3071 - val_accuracy: 0.8639
Epoch 834/1000
341/341 [==============================] - 0s 95us/step - loss: 0.2884 - accuracy: 0.8827 - val_loss: 0

341/341 [==============================] - 0s 91us/step - loss: 0.2858 - accuracy: 0.8827 - val_loss: 0.3070 - val_accuracy: 0.8639
Epoch 884/1000
341/341 [==============================] - 0s 88us/step - loss: 0.2850 - accuracy: 0.8827 - val_loss: 0.3073 - val_accuracy: 0.8639
Epoch 885/1000
341/341 [==============================] - 0s 86us/step - loss: 0.2862 - accuracy: 0.8827 - val_loss: 0.3080 - val_accuracy: 0.8639
Epoch 886/1000
341/341 [==============================] - 0s 86us/step - loss: 0.2860 - accuracy: 0.8827 - val_loss: 0.3070 - val_accuracy: 0.8639
Epoch 887/1000
341/341 [==============================] - 0s 86us/step - loss: 0.2862 - accuracy: 0.8827 - val_loss: 0.3071 - val_accuracy: 0.8639
Epoch 888/1000
341/341 [==============================] - 0s 87us/step - loss: 0.2857 - accuracy: 0.8827 - val_loss: 0.3066 - val_accuracy: 0.8639
Epoch 889/1000
341/341 [==============================] - 0s 89us/step - loss: 0.2848 - accuracy: 0.8827 - val_loss: 0.3105 - val_acc

Epoch 939/1000
341/341 [==============================] - 0s 92us/step - loss: 0.2857 - accuracy: 0.8827 - val_loss: 0.3124 - val_accuracy: 0.8639
Epoch 940/1000
341/341 [==============================] - 0s 90us/step - loss: 0.2861 - accuracy: 0.8827 - val_loss: 0.3076 - val_accuracy: 0.8639
Epoch 941/1000
341/341 [==============================] - 0s 89us/step - loss: 0.2852 - accuracy: 0.8827 - val_loss: 0.3061 - val_accuracy: 0.8639
Epoch 942/1000
341/341 [==============================] - 0s 87us/step - loss: 0.2853 - accuracy: 0.8827 - val_loss: 0.3071 - val_accuracy: 0.8639
Epoch 943/1000
341/341 [==============================] - 0s 92us/step - loss: 0.2849 - accuracy: 0.8827 - val_loss: 0.3064 - val_accuracy: 0.8639
Epoch 944/1000
341/341 [==============================] - 0s 87us/step - loss: 0.2850 - accuracy: 0.8827 - val_loss: 0.3061 - val_accuracy: 0.8639
Epoch 945/1000
341/341 [==============================] - 0s 87us/step - loss: 0.2862 - accuracy: 0.8827 - val_loss: 0

341/341 [==============================] - 0s 108us/step - loss: 0.2845 - accuracy: 0.8827 - val_loss: 0.3064 - val_accuracy: 0.8639
Epoch 995/1000
341/341 [==============================] - 0s 96us/step - loss: 0.2849 - accuracy: 0.8827 - val_loss: 0.3063 - val_accuracy: 0.8639
Epoch 996/1000
341/341 [==============================] - 0s 102us/step - loss: 0.2841 - accuracy: 0.8827 - val_loss: 0.3130 - val_accuracy: 0.8639
Epoch 997/1000
341/341 [==============================] - 0s 101us/step - loss: 0.2861 - accuracy: 0.8827 - val_loss: 0.3117 - val_accuracy: 0.8571
Epoch 998/1000
341/341 [==============================] - 0s 132us/step - loss: 0.2862 - accuracy: 0.8827 - val_loss: 0.3059 - val_accuracy: 0.8639
Epoch 999/1000
341/341 [==============================] - 0s 91us/step - loss: 0.2845 - accuracy: 0.8827 - val_loss: 0.3061 - val_accuracy: 0.8639
Epoch 1000/1000
341/341 [==============================] - 0s 95us/step - loss: 0.2846 - accuracy: 0.8827 - val_loss: 0.3134 - va

In [28]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

147/147 [==============================] - 0s 77us/step
over-sampling test accuracy: 85.71%


In [29]:
############ Feature selection using lasso ##########

In [30]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [31]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [32]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [33]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [34]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 1759
selected features: 31


In [35]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  53,   57,  139,  235,  284,  285,  286,  287,  338,  374,  575,
         618,  640,  688,  700,  760,  776,  805,  866,  944, 1036, 1254,
        1273, 1289, 1460, 1481, 1547, 1609, 1638, 1757, 1758]])

In [36]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTATGTATGA', 'TTTTATATACTTTA', 'TTTAATATTATAAAATG',
       'TTCGTATTTAGC',
       'TTATTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGA',
       'TTATTATGTTAATG',
       'TTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAA',
       'TTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGC',
       'TTACTATATTTGGCATTATATGTAGTATTATTTTCACGAGATGGG', 'TTAACATAATAAT',
       'TCATAACACAAGAGATATAGCAAAGTTTAAAAAAGTACTATAAAATAGCAATTGAATGTCCAGTAACAAATTTGGAGGAAGCGTATATGTATCAAACAAT',
       'TATTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGAT',
       'TATTAATCATGAAC',
       'TATAAGGTCATGTCACGTTGAAACAGAGGTTCCTCGGCATTTTTGAAAATAATCATTAGCTTGCTCACTATATAAT',
       'TAGTGAGCAAG',
       'TAATTGTTTGATACATATACGCTTCCTCCAAATTTGTTACTGGACATTCAATTGCTATTTTATAGTACTTTTTTAAACTTTGCTATATCTCTTGTGTTAT',
       'TAATAATTGTTTGATACATATACGCTTCCTCCAAATTTGTTACTGGACATTCAATTGCTA

In [38]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756],
           dtype='int64', length=1728)

In [39]:
len(removed_features)

1728

In [40]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 31) (255,) (255, 32)


In [41]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 219), (1, 197)]


In [42]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 244), (1, 244)]


In [43]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [45]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [46]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [47]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 291 samples, validate on 125 samples
Epoch 1/1000
291/291 [==============================] - 0s 373us/step - loss: 35.9494 - accuracy: 0.5292 - val_loss: 0.6122 - val_accuracy: 0.5520
Epoch 2/1000
291/291 [==============================] - 0s 64us/step - loss: 0.6016 - accuracy: 0.7251 - val_loss: 0.5852 - val_accuracy: 0.8480
Epoch 3/1000
291/291 [==============================] - 0s 92us/step - loss: 0.5851 - accuracy: 0.8179 - val_loss: 0.5715 - val_accuracy: 0.8480
Epoch 4/1000
291/291 [==============================] - 0s 82us/step - loss: 0.5740 - accuracy: 0.8179 - val_loss: 0.5641 - val_accuracy: 0.8480
Epoch 5/1000
291/291 [==============================] - 0s 96us/step - loss: 0.5699 - accuracy: 0.8247 - val_loss: 0.5521 - val_accuracy: 0.8560
Epoch 6/1000
291/291 [==============================] - 0s 91us/step - loss: 0.5598 - accuracy: 0.8179 - val_loss: 0.5419 - val_accuracy: 0.8560
Epoch 7/1000
291/291 [==============================] - 0s 157us/step - loss: 0.55

291/291 [==============================] - 0s 73us/step - loss: 0.3999 - accuracy: 0.8316 - val_loss: 0.3693 - val_accuracy: 0.8640
Epoch 57/1000
291/291 [==============================] - 0s 70us/step - loss: 0.4003 - accuracy: 0.8316 - val_loss: 0.3673 - val_accuracy: 0.8560
Epoch 58/1000
291/291 [==============================] - 0s 76us/step - loss: 0.3984 - accuracy: 0.8282 - val_loss: 0.3682 - val_accuracy: 0.8560
Epoch 59/1000
291/291 [==============================] - 0s 91us/step - loss: 0.3989 - accuracy: 0.8316 - val_loss: 0.3662 - val_accuracy: 0.8560
Epoch 60/1000
291/291 [==============================] - 0s 64us/step - loss: 0.3984 - accuracy: 0.8282 - val_loss: 0.3647 - val_accuracy: 0.8560
Epoch 61/1000
291/291 [==============================] - 0s 71us/step - loss: 0.3992 - accuracy: 0.8282 - val_loss: 0.3629 - val_accuracy: 0.8560
Epoch 62/1000
291/291 [==============================] - 0s 81us/step - loss: 0.3949 - accuracy: 0.8282 - val_loss: 0.3615 - val_accuracy:

291/291 [==============================] - 0s 115us/step - loss: 0.3656 - accuracy: 0.8454 - val_loss: 0.3299 - val_accuracy: 0.8800
Epoch 113/1000
291/291 [==============================] - 0s 139us/step - loss: 0.3640 - accuracy: 0.8488 - val_loss: 0.3318 - val_accuracy: 0.8800
Epoch 114/1000
291/291 [==============================] - 0s 133us/step - loss: 0.3641 - accuracy: 0.8454 - val_loss: 0.3284 - val_accuracy: 0.8800
Epoch 115/1000
291/291 [==============================] - 0s 100us/step - loss: 0.3653 - accuracy: 0.8488 - val_loss: 0.3283 - val_accuracy: 0.8800
Epoch 116/1000
291/291 [==============================] - 0s 125us/step - loss: 0.3639 - accuracy: 0.8454 - val_loss: 0.3283 - val_accuracy: 0.8800
Epoch 117/1000
291/291 [==============================] - 0s 105us/step - loss: 0.3633 - accuracy: 0.8454 - val_loss: 0.3282 - val_accuracy: 0.8800
Epoch 118/1000
291/291 [==============================] - 0s 104us/step - loss: 0.3610 - accuracy: 0.8454 - val_loss: 0.3276 - 

291/291 [==============================] - 0s 88us/step - loss: 0.3505 - accuracy: 0.8488 - val_loss: 0.3150 - val_accuracy: 0.8800
Epoch 168/1000
291/291 [==============================] - 0s 70us/step - loss: 0.3511 - accuracy: 0.8488 - val_loss: 0.3144 - val_accuracy: 0.8800
Epoch 169/1000
291/291 [==============================] - 0s 86us/step - loss: 0.3506 - accuracy: 0.8522 - val_loss: 0.3137 - val_accuracy: 0.8800
Epoch 170/1000
291/291 [==============================] - 0s 71us/step - loss: 0.3504 - accuracy: 0.8522 - val_loss: 0.3131 - val_accuracy: 0.8800
Epoch 171/1000
291/291 [==============================] - 0s 75us/step - loss: 0.3496 - accuracy: 0.8522 - val_loss: 0.3262 - val_accuracy: 0.8720
Epoch 172/1000
291/291 [==============================] - 0s 68us/step - loss: 0.3567 - accuracy: 0.8488 - val_loss: 0.3135 - val_accuracy: 0.8800
Epoch 173/1000
291/291 [==============================] - 0s 64us/step - loss: 0.3502 - accuracy: 0.8488 - val_loss: 0.3127 - val_acc

291/291 [==============================] - 0s 110us/step - loss: 0.3449 - accuracy: 0.8522 - val_loss: 0.3052 - val_accuracy: 0.8800
Epoch 223/1000
291/291 [==============================] - 0s 87us/step - loss: 0.3423 - accuracy: 0.8557 - val_loss: 0.3050 - val_accuracy: 0.8800
Epoch 224/1000
291/291 [==============================] - 0s 126us/step - loss: 0.3424 - accuracy: 0.8522 - val_loss: 0.3049 - val_accuracy: 0.8800
Epoch 225/1000
291/291 [==============================] - 0s 93us/step - loss: 0.3405 - accuracy: 0.8591 - val_loss: 0.3042 - val_accuracy: 0.8880
Epoch 226/1000
291/291 [==============================] - 0s 77us/step - loss: 0.3435 - accuracy: 0.8557 - val_loss: 0.3040 - val_accuracy: 0.8800
Epoch 227/1000
291/291 [==============================] - 0s 77us/step - loss: 0.3410 - accuracy: 0.8591 - val_loss: 0.3036 - val_accuracy: 0.8880
Epoch 228/1000
291/291 [==============================] - 0s 70us/step - loss: 0.3422 - accuracy: 0.8522 - val_loss: 0.3039 - val_a

...

Epoch 777/1000
291/291 [==============================] - 0s 178us/step - loss: 0.3260 - accuracy: 0.8625 - val_loss: 0.2882 - val_accuracy: 0.8880
Epoch 778/1000
291/291 [==============================] - 0s 139us/step - loss: 0.3264 - accuracy: 0.8660 - val_loss: 0.2882 - val_accuracy: 0.8880
Epoch 779/1000
291/291 [==============================] - 0s 129us/step - loss: 0.3253 - accuracy: 0.8660 - val_loss: 0.2882 - val_accuracy: 0.8880
Epoch 780/1000
291/291 [==============================] - 0s 93us/step - loss: 0.3263 - accuracy: 0.8660 - val_loss: 0.2882 - val_accuracy: 0.8880
Epoch 781/1000
291/291 [==============================] - 0s 107us/step - loss: 0.3252 - accuracy: 0.8660 - val_loss: 0.2888 - val_accuracy: 0.8880
Epoch 782/1000
291/291 [==============================] - 0s 117us/step - loss: 0.3287 - accuracy: 0.8625 - val_loss: 0.2885 - val_accuracy: 0.8880
Epoch 783/1000
291/291 [==============================] - 0s 135us/step - loss: 0.3255 - accuracy: 0.8660 - val_l

291/291 [==============================] - 0s 142us/step - loss: 0.3275 - accuracy: 0.8625 - val_loss: 0.2887 - val_accuracy: 0.8880
Epoch 833/1000
291/291 [==============================] - 0s 155us/step - loss: 0.3266 - accuracy: 0.8660 - val_loss: 0.2880 - val_accuracy: 0.8880
Epoch 834/1000
291/291 [==============================] - 0s 153us/step - loss: 0.3253 - accuracy: 0.8660 - val_loss: 0.2881 - val_accuracy: 0.8880
Epoch 835/1000
291/291 [==============================] - 0s 109us/step - loss: 0.3253 - accuracy: 0.8660 - val_loss: 0.2885 - val_accuracy: 0.8880
Epoch 836/1000
291/291 [==============================] - 0s 190us/step - loss: 0.3253 - accuracy: 0.8660 - val_loss: 0.2879 - val_accuracy: 0.8880
Epoch 837/1000
291/291 [==============================] - 0s 104us/step - loss: 0.3256 - accuracy: 0.8660 - val_loss: 0.2883 - val_accuracy: 0.8880
Epoch 838/1000
291/291 [==============================] - 0s 93us/step - loss: 0.3253 - accuracy: 0.8660 - val_loss: 0.2879 - v

291/291 [==============================] - 0s 135us/step - loss: 0.3259 - accuracy: 0.8660 - val_loss: 0.2879 - val_accuracy: 0.8880
Epoch 888/1000
291/291 [==============================] - 0s 148us/step - loss: 0.3245 - accuracy: 0.8660 - val_loss: 0.2878 - val_accuracy: 0.8880
Epoch 889/1000
291/291 [==============================] - 0s 120us/step - loss: 0.3248 - accuracy: 0.8660 - val_loss: 0.2882 - val_accuracy: 0.8880
Epoch 890/1000
291/291 [==============================] - 0s 113us/step - loss: 0.3258 - accuracy: 0.8660 - val_loss: 0.2881 - val_accuracy: 0.8880
Epoch 891/1000
291/291 [==============================] - 0s 154us/step - loss: 0.3254 - accuracy: 0.8660 - val_loss: 0.2891 - val_accuracy: 0.8880
Epoch 892/1000
291/291 [==============================] - 0s 124us/step - loss: 0.3289 - accuracy: 0.8625 - val_loss: 0.2882 - val_accuracy: 0.8880
Epoch 893/1000
291/291 [==============================] - 0s 120us/step - loss: 0.3252 - accuracy: 0.8660 - val_loss: 0.2880 - 

291/291 [==============================] - 0s 124us/step - loss: 0.3247 - accuracy: 0.8660 - val_loss: 0.2885 - val_accuracy: 0.8880
Epoch 943/1000
291/291 [==============================] - 0s 196us/step - loss: 0.3252 - accuracy: 0.8660 - val_loss: 0.3236 - val_accuracy: 0.8640
Epoch 944/1000
291/291 [==============================] - 0s 99us/step - loss: 0.3522 - accuracy: 0.8419 - val_loss: 0.2920 - val_accuracy: 0.8880
Epoch 945/1000
291/291 [==============================] - 0s 107us/step - loss: 0.3287 - accuracy: 0.8625 - val_loss: 0.2895 - val_accuracy: 0.8880
Epoch 946/1000
291/291 [==============================] - 0s 88us/step - loss: 0.3259 - accuracy: 0.8660 - val_loss: 0.2886 - val_accuracy: 0.8880
Epoch 947/1000
291/291 [==============================] - 0s 97us/step - loss: 0.3254 - accuracy: 0.8660 - val_loss: 0.2884 - val_accuracy: 0.8880
Epoch 948/1000
291/291 [==============================] - 0s 108us/step - loss: 0.3251 - accuracy: 0.8660 - val_loss: 0.2884 - val

291/291 [==============================] - 0s 77us/step - loss: 0.3255 - accuracy: 0.8660 - val_loss: 0.2883 - val_accuracy: 0.8880
Epoch 998/1000
291/291 [==============================] - 0s 95us/step - loss: 0.3243 - accuracy: 0.8660 - val_loss: 0.2885 - val_accuracy: 0.8880
Epoch 999/1000
291/291 [==============================] - 0s 102us/step - loss: 0.3264 - accuracy: 0.8660 - val_loss: 0.2881 - val_accuracy: 0.8880
Epoch 1000/1000
291/291 [==============================] - 0s 139us/step - loss: 0.3240 - accuracy: 0.8660 - val_loss: 0.2882 - val_accuracy: 0.8880


In [48]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


125/125 [==============================] - 0s 62us/step
combination test accuracy: 88.80%


In [49]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [50]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 341 samples, validate on 147 samples
Epoch 1/1000
341/341 [==============================] - 0s 374us/step - loss: 77.1837 - accuracy: 0.6246 - val_loss: 0.6566 - val_accuracy: 0.6803
Epoch 2/1000
341/341 [==============================] - 0s 68us/step - loss: 0.6465 - accuracy: 0.7243 - val_loss: 0.6452 - val_accuracy: 0.6803
Epoch 3/1000
341/341 [==============================] - 0s 71us/step - loss: 0.6432 - accuracy: 0.7185 - val_loss: 0.6477 - val_accuracy: 0.6735
Epoch 4/1000
341/341 [==============================] - 0s 75us/step - loss: 0.6331 - accuracy: 0.7243 - val_loss: 0.6448 - val_accuracy: 0.6735
Epoch 5/1000
341/341 [==============================] - 0s 111us/step - loss: 0.6317 - accuracy: 0.7243 - val_loss: 0.6256 - val_accuracy: 0.7347
Epoch 6/1000
341/341 [==============================] - 0s 124us/step - loss: 0.6344 - accuracy: 0.7185 - val_loss: 0.6231 - val_accuracy: 0.6735
Epoch 7/1000
341/341 [==============================] - 0s 168us/step - loss: 0.

341/341 [==============================] - 0s 95us/step - loss: 0.5101 - accuracy: 0.7830 - val_loss: 0.5430 - val_accuracy: 0.7211
Epoch 57/1000
341/341 [==============================] - 0s 99us/step - loss: 0.5114 - accuracy: 0.7801 - val_loss: 0.5118 - val_accuracy: 0.7823
Epoch 58/1000
341/341 [==============================] - 0s 111us/step - loss: 0.5080 - accuracy: 0.7889 - val_loss: 0.5110 - val_accuracy: 0.7823
Epoch 59/1000
341/341 [==============================] - 0s 107us/step - loss: 0.5081 - accuracy: 0.7830 - val_loss: 0.5110 - val_accuracy: 0.7823
Epoch 60/1000
341/341 [==============================] - 0s 114us/step - loss: 0.5101 - accuracy: 0.7859 - val_loss: 0.5101 - val_accuracy: 0.7823
Epoch 61/1000
341/341 [==============================] - 0s 102us/step - loss: 0.5063 - accuracy: 0.7801 - val_loss: 0.5084 - val_accuracy: 0.7823
Epoch 62/1000
341/341 [==============================] - 0s 199us/step - loss: 0.5097 - accuracy: 0.7771 - val_loss: 0.5064 - val_accu

Epoch 112/1000
341/341 [==============================] - 0s 90us/step - loss: 0.4695 - accuracy: 0.7977 - val_loss: 0.5045 - val_accuracy: 0.7279
Epoch 113/1000
341/341 [==============================] - 0s 92us/step - loss: 0.4766 - accuracy: 0.7713 - val_loss: 0.4887 - val_accuracy: 0.7279
Epoch 114/1000
341/341 [==============================] - 0s 99us/step - loss: 0.4711 - accuracy: 0.7859 - val_loss: 0.4674 - val_accuracy: 0.7891
Epoch 115/1000
341/341 [==============================] - 0s 126us/step - loss: 0.4742 - accuracy: 0.7947 - val_loss: 0.4853 - val_accuracy: 0.7619
Epoch 116/1000
341/341 [==============================] - 0s 107us/step - loss: 0.4724 - accuracy: 0.7830 - val_loss: 0.4677 - val_accuracy: 0.7891
Epoch 117/1000
341/341 [==============================] - 0s 113us/step - loss: 0.4653 - accuracy: 0.7889 - val_loss: 0.4683 - val_accuracy: 0.7891
Epoch 118/1000
341/341 [==============================] - 0s 91us/step - loss: 0.4673 - accuracy: 0.7947 - val_loss

341/341 [==============================] - 0s 118us/step - loss: 0.4415 - accuracy: 0.7918 - val_loss: 0.4589 - val_accuracy: 0.7619
Epoch 168/1000
341/341 [==============================] - 0s 128us/step - loss: 0.4456 - accuracy: 0.7859 - val_loss: 0.4596 - val_accuracy: 0.7959
Epoch 169/1000
341/341 [==============================] - 0s 78us/step - loss: 0.4411 - accuracy: 0.7918 - val_loss: 0.4423 - val_accuracy: 0.7891
Epoch 170/1000
341/341 [==============================] - 0s 73us/step - loss: 0.4403 - accuracy: 0.7918 - val_loss: 0.4378 - val_accuracy: 0.7891
Epoch 171/1000
341/341 [==============================] - 0s 83us/step - loss: 0.4442 - accuracy: 0.7859 - val_loss: 0.4599 - val_accuracy: 0.7619
Epoch 172/1000
341/341 [==============================] - 0s 102us/step - loss: 0.4446 - accuracy: 0.7918 - val_loss: 0.4638 - val_accuracy: 0.7619
Epoch 173/1000
341/341 [==============================] - 0s 112us/step - loss: 0.4471 - accuracy: 0.7859 - val_loss: 0.4359 - val

341/341 [==============================] - 0s 107us/step - loss: 0.4190 - accuracy: 0.7859 - val_loss: 0.4220 - val_accuracy: 0.7891
Epoch 223/1000
341/341 [==============================] - 0s 114us/step - loss: 0.4289 - accuracy: 0.7889 - val_loss: 0.4317 - val_accuracy: 0.8231
Epoch 224/1000
341/341 [==============================] - 0s 82us/step - loss: 0.4293 - accuracy: 0.7977 - val_loss: 0.4183 - val_accuracy: 0.7891
Epoch 225/1000
341/341 [==============================] - 0s 74us/step - loss: 0.4201 - accuracy: 0.7918 - val_loss: 0.4246 - val_accuracy: 0.7891
Epoch 226/1000
341/341 [==============================] - 0s 64us/step - loss: 0.4147 - accuracy: 0.7947 - val_loss: 0.4813 - val_accuracy: 0.7347
Epoch 227/1000
341/341 [==============================] - 0s 64us/step - loss: 0.4311 - accuracy: 0.7830 - val_loss: 0.4215 - val_accuracy: 0.7959
Epoch 228/1000
341/341 [==============================] - 0s 62us/step - loss: 0.4122 - accuracy: 0.7977 - val_loss: 0.5062 - val_a

...

341/341 [==============================] - 0s 106us/step - loss: 0.3711 - accuracy: 0.8328 - val_loss: 0.3937 - val_accuracy: 0.7687
Epoch 775/1000
341/341 [==============================] - 0s 84us/step - loss: 0.3655 - accuracy: 0.8328 - val_loss: 0.3745 - val_accuracy: 0.8299
Epoch 776/1000
341/341 [==============================] - 0s 93us/step - loss: 0.3666 - accuracy: 0.8358 - val_loss: 0.3765 - val_accuracy: 0.8299
Epoch 777/1000
341/341 [==============================] - 0s 96us/step - loss: 0.3670 - accuracy: 0.8270 - val_loss: 0.3707 - val_accuracy: 0.8367
Epoch 778/1000
341/341 [==============================] - 0s 99us/step - loss: 0.3591 - accuracy: 0.8387 - val_loss: 0.3697 - val_accuracy: 0.8367
Epoch 779/1000
341/341 [==============================] - 0s 139us/step - loss: 0.3609 - accuracy: 0.8416 - val_loss: 0.3725 - val_accuracy: 0.8367
Epoch 780/1000
341/341 [==============================] - 0s 118us/step - loss: 0.3669 - accuracy: 0.8328 - val_loss: 0.3741 - val_

341/341 [==============================] - 0s 163us/step - loss: 0.3600 - accuracy: 0.8358 - val_loss: 0.3985 - val_accuracy: 0.7687
Epoch 830/1000
341/341 [==============================] - 0s 126us/step - loss: 0.3662 - accuracy: 0.8299 - val_loss: 0.3744 - val_accuracy: 0.8299
Epoch 831/1000
341/341 [==============================] - 0s 123us/step - loss: 0.3605 - accuracy: 0.8358 - val_loss: 0.3699 - val_accuracy: 0.8367
Epoch 832/1000
341/341 [==============================] - 0s 91us/step - loss: 0.3574 - accuracy: 0.8416 - val_loss: 0.4433 - val_accuracy: 0.7687
Epoch 833/1000
341/341 [==============================] - 0s 67us/step - loss: 0.3631 - accuracy: 0.8387 - val_loss: 0.6274 - val_accuracy: 0.7687
Epoch 834/1000
341/341 [==============================] - 0s 103us/step - loss: 0.4150 - accuracy: 0.8182 - val_loss: 0.3930 - val_accuracy: 0.8231
Epoch 835/1000
341/341 [==============================] - 0s 108us/step - loss: 0.3626 - accuracy: 0.8358 - val_loss: 0.3711 - va

Epoch 885/1000
341/341 [==============================] - 0s 68us/step - loss: 0.3581 - accuracy: 0.8416 - val_loss: 0.3722 - val_accuracy: 0.8299
Epoch 886/1000
341/341 [==============================] - 0s 59us/step - loss: 0.3699 - accuracy: 0.8299 - val_loss: 0.3852 - val_accuracy: 0.8299
Epoch 887/1000
341/341 [==============================] - 0s 72us/step - loss: 0.3745 - accuracy: 0.8328 - val_loss: 0.4210 - val_accuracy: 0.7959
Epoch 888/1000
341/341 [==============================] - 0s 63us/step - loss: 0.3741 - accuracy: 0.8270 - val_loss: 0.3802 - val_accuracy: 0.8299
Epoch 889/1000
341/341 [==============================] - 0s 60us/step - loss: 0.3636 - accuracy: 0.8328 - val_loss: 0.3723 - val_accuracy: 0.8299
Epoch 890/1000
341/341 [==============================] - 0s 62us/step - loss: 0.3587 - accuracy: 0.8387 - val_loss: 0.3689 - val_accuracy: 0.8367
Epoch 891/1000
341/341 [==============================] - 0s 60us/step - loss: 0.3589 - accuracy: 0.8387 - val_loss: 0

Epoch 941/1000
341/341 [==============================] - 0s 70us/step - loss: 0.3562 - accuracy: 0.8416 - val_loss: 0.3736 - val_accuracy: 0.8367
Epoch 942/1000
341/341 [==============================] - 0s 70us/step - loss: 0.3591 - accuracy: 0.8387 - val_loss: 0.5562 - val_accuracy: 0.7687
Epoch 943/1000
341/341 [==============================] - 0s 54us/step - loss: 0.3928 - accuracy: 0.8211 - val_loss: 0.3695 - val_accuracy: 0.8367
Epoch 944/1000
341/341 [==============================] - 0s 58us/step - loss: 0.3584 - accuracy: 0.8358 - val_loss: 0.3732 - val_accuracy: 0.8299
Epoch 945/1000
341/341 [==============================] - 0s 65us/step - loss: 0.3639 - accuracy: 0.8299 - val_loss: 0.3799 - val_accuracy: 0.8299
Epoch 946/1000
341/341 [==============================] - 0s 77us/step - loss: 0.3618 - accuracy: 0.8328 - val_loss: 0.3950 - val_accuracy: 0.7687
Epoch 947/1000
341/341 [==============================] - 0s 61us/step - loss: 0.3610 - accuracy: 0.8328 - val_loss: 0

341/341 [==============================] - 0s 80us/step - loss: 0.3598 - accuracy: 0.8358 - val_loss: 0.3747 - val_accuracy: 0.8299
Epoch 997/1000
341/341 [==============================] - 0s 92us/step - loss: 0.3553 - accuracy: 0.8387 - val_loss: 0.3741 - val_accuracy: 0.8299
Epoch 998/1000
341/341 [==============================] - 0s 101us/step - loss: 0.3578 - accuracy: 0.8358 - val_loss: 0.3741 - val_accuracy: 0.8299
Epoch 999/1000
341/341 [==============================] - 0s 123us/step - loss: 0.3579 - accuracy: 0.8328 - val_loss: 0.3714 - val_accuracy: 0.8367
Epoch 1000/1000
341/341 [==============================] - 0s 141us/step - loss: 0.3585 - accuracy: 0.8387 - val_loss: 0.3776 - val_accuracy: 0.8299


In [52]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

147/147 [==============================] - 0s 195us/step
combination test accuracy: 82.99%


In [53]:
############ Logistic Regression ############

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [55]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [57]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 100.00%


In [59]:
############## Random Forest ##############

In [60]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [61]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [62]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [63]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [64]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [65]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         1.         1.         0.98412698]
0.9968253968253968


In [66]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98550725 1.         0.98529412 0.98529412 0.98529412]
0.9882779198635976
